# Credit Risk Resampling Techniques

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import os

# Read the CSV and Perform Basic Data Cleaning

In [4]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
print(file_path)
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head(100)

Resources\LoanStats_2019Q1.csv


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20000.0,0.0881,634.23,MORTGAGE,115000.0,Not Verified,Mar-2019,low_risk,n,32.58,...,96.8,0.0,0.0,0.0,231097.0,139116.0,67300.0,135997.0,N,N
96,24000.0,0.1240,538.74,RENT,50000.0,Source Verified,Mar-2019,low_risk,n,25.88,...,100.0,0.0,0.0,0.0,92477.0,40225.0,32900.0,52577.0,N,N
97,14000.0,0.1390,325.04,RENT,60000.0,Not Verified,Mar-2019,low_risk,n,11.36,...,94.1,100.0,0.0,0.0,57207.0,51341.0,10300.0,40007.0,N,N
98,40000.0,0.0702,1235.45,MORTGAGE,150000.0,Source Verified,Mar-2019,low_risk,n,13.66,...,100.0,72.7,0.0,0.0,323723.0,52914.0,42900.0,24409.0,N,N


# Split the Data into Training and Testing

In [6]:
# Create our features
catagoryColumns = ['home_ownership','verification_status', 'pymnt_plan', 'hardship_flag', 'issue_d','debt_settlement_flag']

y = df[['loan_status']]
print(y.head())

X= df
del X['loan_status']


  loan_status
0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk


In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [8]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [10]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampledROS, y_resampledROS = ros.fit_resample(X_train, y_train)

Counter(y_resampledROS)

Counter({'loan_status': 1})

In [11]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampledROS, y_resampledROS)
predictions = model.predict(X_test)


from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)


0.6301075268817204

In [12]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)


0.6663237827524566

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, predictions)
print(matrix)

print(classification_report(y_test, predictions))


[[   71    30]
 [ 6334 10770]]
              precision    recall  f1-score   support

   high_risk       0.01      0.70      0.02       101
    low_risk       1.00      0.63      0.77     17104

    accuracy                           0.63     17205
   macro avg       0.50      0.67      0.40     17205
weighted avg       0.99      0.63      0.77     17205



In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.63      0.02      0.67      0.45       101
   low_risk       1.00      0.63      0.70      0.77      0.67      0.44     17104

avg / total       0.99      0.63      0.70      0.77      0.67      0.44     17205



### SMOTE Oversampling

In [15]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_resampledSMOTE, y_resampledSMOTE = smote.fit_resample(X_train, y_train)


In [16]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampledSMOTE, y_resampledSMOTE)
SMOTE_predictions = model.predict(X_test)

In [17]:
# Calculated the balanced accuracy score
print(balanced_accuracy_score(y_test, SMOTE_predictions))


0.6429238369346757


In [18]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, SMOTE_predictions)
print(matrix)

print(classification_report(y_test, SMOTE_predictions))


[[   61    40]
 [ 5441 11663]]
              precision    recall  f1-score   support

   high_risk       0.01      0.60      0.02       101
    low_risk       1.00      0.68      0.81     17104

    accuracy                           0.68     17205
   macro avg       0.50      0.64      0.42     17205
weighted avg       0.99      0.68      0.81     17205



In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, SMOTE_predictions))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.60      0.68      0.02      0.64      0.41       101
   low_risk       1.00      0.68      0.60      0.81      0.64      0.42     17104

avg / total       0.99      0.68      0.60      0.81      0.64      0.42     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [20]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampledCC, y_resampledCC = cc.fit_resample(X_train, y_train)


Counter(y_resampledCC)

# YOUR CODE HERE

Counter({'loan_status': 1})

In [21]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampledCC, y_resampledCC)
CC_predictions = model.predict(X_test)

In [22]:
# Calculated the balanced accuracy score
print(balanced_accuracy_score(y_test, CC_predictions))


0.5442661782548694


In [23]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, CC_predictions)
print(matrix)
print(classification_report(y_test, CC_predictions))


[[   70    31]
 [10340  6764]]
              precision    recall  f1-score   support

   high_risk       0.01      0.69      0.01       101
    low_risk       1.00      0.40      0.57     17104

    accuracy                           0.40     17205
   macro avg       0.50      0.54      0.29     17205
weighted avg       0.99      0.40      0.56     17205



In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, CC_predictions))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.40      0.01      0.52      0.28       101
   low_risk       1.00      0.40      0.69      0.57      0.52      0.27     17104

avg / total       0.99      0.40      0.69      0.56      0.52      0.27     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [90]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to completefrom imblearn.combine import SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampledSMOTEENN, y_resampledSMOTEENN = smote_enn.fit_resample(X, y)
Counter(y_resampledSMOTEENN)



Counter({'loan_status': 1})

In [91]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampledSMOTEENN, y_resampledSMOTEENN)
SMOTEENN_predictions = model.predict(X_test)

In [92]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
print(balanced_accuracy_score(y_test, SMOTEENN_predictions))


0.6447701423556762


In [98]:
# Display the confusion matrix
# YOUR CODE HERE
matrix = confusion_matrix(y_test, SMOTEENN_predictions)
print(matrix)
print(classification_report(y_test, SMOTEENN_predictions))



[[  73   28]
 [7410 9694]]
              precision    recall  f1-score   support

   high_risk       0.01      0.72      0.02       101
    low_risk       1.00      0.57      0.72     17104

    accuracy                           0.57     17205
   macro avg       0.50      0.64      0.37     17205
weighted avg       0.99      0.57      0.72     17205



In [94]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, SMOTEENN_predictions))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.72      0.57      0.02      0.64      0.42       101
   low_risk       1.00      0.57      0.72      0.72      0.64      0.40     17104

avg / total       0.99      0.57      0.72      0.72      0.64      0.40     17205



# Analysis and Conclusions
We tried several models with different over and sampling methods.  None of the the models where usable with low .01 precision on all tools and Recall less then .75 on all models for high risk judegements

## Oversampling
The Naive Random Oversamples and Smote oversampling methods produced unworkable models.  With the Native oversampling performing slightly better with a recall of 0.70 versus 0.60.  Both precisions were terrible at 0.01

## Undersampling
ClusterCentroids undersampling methods were similiar to the Native Random Oversample results with a recall of 0.69 and a precision of 0.01

## Combination
The Combitation sampling had the best recall of 0.72 and but still abymal 0.01 precision scores and a low f2 score of 0.02.
